# `SELECT`

Nesta aula vamos explorar mais sobre o comando `SELECT`, usado para consultar a base de dados. 

Instale a base de teste 'musica' usando o script `musica.sql`. Este script instala a base de dados de exemplo do livro-texto da disciplina, de modo que vocês podem praticar com os exercícios do livro também!

Eis aqui o diagrama do modelo relacional dessa base de dados:

<center><img src="imgs/modelo_relacional.png" width=800/></center>

Eis a lista completa de chaves primárias e estrangeiras:

```SQL
ALTER TABLE AUTOR
    ADD PRIMARY KEY (Codigo_Autor);

ALTER TABLE CD
    ADD PRIMARY KEY (Codigo_CD) ;

ALTER TABLE GRAVADORA
    ADD PRIMARY KEY (Codigo_Gravadora) ;

ALTER TABLE FAIXA
    ADD PRIMARY KEY (Codigo_Musica, Codigo_CD);

ALTER TABLE MUSICA
    ADD PRIMARY KEY (Codigo_Musica) ;

ALTER TABLE MUSICA_AUTOR
    ADD PRIMARY KEY (Codigo_Musica, Codigo_Autor);

ALTER TABLE CD
    ADD FOREIGN KEY (Codigo_Gravadora)
        REFERENCES GRAVADORA(Codigo_Gravadora);

ALTER TABLE CD
    ADD FOREIGN KEY (CD_Indicado)
        REFERENCES CD(Codigo_CD);

ALTER TABLE FAIXA
    ADD FOREIGN KEY (Codigo_CD)
        REFERENCES CD(Codigo_CD);

ALTER TABLE FAIXA
    ADD FOREIGN KEY (Codigo_Musica)
        REFERENCES MUSICA(Codigo_Musica);

ALTER TABLE MUSICA_AUTOR
    ADD FOREIGN KEY (Codigo_Autor)
        REFERENCES AUTOR(Codigo_Autor);

ALTER TABLE MUSICA_AUTOR
    ADD FOREIGN KEY (Codigo_Musica)
        REFERENCES MUSICA(Codigo_Musica);
```

**Atividade:** Quais destas tabelas são tabelas de relacionamento?

Música_autor e faixa.

Vamos agora criar a conexão com o banco de dados e o objeto auxiliar de conexão. Aqui resolvi usar uma estratégia diferente de implementação que dá o mesmo resultado, para deixar de exemplo para vocês:

In [60]:
from functools import partial
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='megadados',
    password='megadados',
    database='musica',
)


def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)


db = partial(run_db_query, connection)

## Explorando a estrutura da base de dados

Agora podemos usar nosso helper para enviar comandos à base de dados. Vamos ver quais tabelas existem na base 'musica':

In [61]:
db('SHOW TABLES')

Executando query:
('autor',)
('cd',)
('cd_categoria',)
('faixa',)
('gravadora',)
('musica',)
('musica_autor',)


Para saber qual o schema da tabela 'cd', podemos usar o comando '`DESCRIBE`'

In [62]:
db('DESCRIBE CD')

Executando query:
('Codigo_CD', b'int', 'NO', 'PRI', None, '')
('Codigo_Gravadora', b'int', 'YES', 'MUL', None, '')
('Nome_CD', b'varchar(60)', 'YES', '', None, '')
('Preco_Venda', b'int', 'YES', '', None, '')
('Data_Lancamento', b'date', 'YES', '', None, '')
('CD_Indicado', b'int', 'YES', 'MUL', None, '')


## Consultando a base de dados

Relembrando: vamos usar o comando '`SELECT`' para listar os conteudos da tabela 'cd'

In [63]:
db('SELECT * FROM CD')

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)


O comando acima lista todos os registros da tabela 'cd', com todas as colunas presentes:

![Seleção da tabela inteira](imgs/tudo.png)

Vamos agora selecionar apenas algumas colunas para exibir.

In [64]:
db('SELECT Nome_CD, Data_Lancamento FROM CD')

Executando query:
('Mais do Mesmo', datetime.date(1998, 10, 1))
('Bate-Boca', datetime.date(1999, 7, 1))
('Elis Regina - Essa Mulher', datetime.date(1989, 5, 1))
('A Força que nunca Seca', datetime.date(1998, 12, 1))
('Perfil', datetime.date(2001, 5, 1))
('Barry Manilow Greatest Hits Vol I', datetime.date(1991, 11, 1))
('Listen Without Prejudice', datetime.date(1991, 10, 1))


Agora vemos apenas as colunas escolhidas. Relembrando: a operação de seleção de colunas chama-se **projeção**.

![Projeção](imgs/projecao.png)

Vamos agora atuar na escolha de linhas, selecionando quais desejamos. Para escolher todas as linhas cujo Nome_CD terminem em 'a', podemos executar a query a seguir:

In [65]:
db("SELECT * FROM CD WHERE Nome_CD LIKE '%a'")

Executando query:
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)


Vemos apenas as linhas escolhidas. Recordando: a operação de filtragem de linhas apropriadas chama-se **seleção**.

![Seleção](imgs/selecao.png)

## Cláusula `WHERE`

A cláusula `WHERE` permite filtrar as linhas da tabela (ou tabelas - mais sobre *JOIN* daqui a pouco). Basta especificar a condição de filtragem: o resultado da query será o conjunto de linhas para as quais a condição é verdadeira.

Já vimos alguns usos da cláusula `WHERE` acima. Vamos ver mais exemplos:

In [66]:
# Queries equivalentes.
db('SELECT * FROM CD')
db('SELECT * FROM CD WHERE True')

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)
Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)


Qual o cd mais antigo que custa 13 reais ou mais?

In [67]:
db('''
SELECT 
    Nome_CD 
FROM 
    CD
WHERE 
    Preco_venda >= 13 
ORDER BY 
    Data_Lancamento ASC 
LIMIT 1
''')

Executando query:
('Elis Regina - Essa Mulher',)


### Vamos praticar queries!

Preencha os códigos a seguir com buscas adequadas. Consulte seu livro texto para descobrir como montar essas queries.

Qual o nome da música mais longa?

In [68]:
#db(''' SELECT * FROM MUSICA''')
db('''DESCRIBE MUSICA''')

Executando query:
('Codigo_Musica', b'int', 'NO', 'PRI', None, '')
('Nome_Musica', b'varchar(60)', 'YES', '', None, '')
('Duracao', b'int', 'YES', '', None, '')


In [69]:
db('''
SELECT 
    Nome_Musica 
FROM 
    MUSICA
ORDER BY 
    Duracao DESC
LIMIT 1
''')

Executando query:
('Faroeste Caboclo',)


Quais gravadoras não tem endereço declarado?

In [70]:
db('''
SELECT 
    Nome_Gravadora 
FROM 
    GRAVADORA
WHERE 
    Endereco IS NULL
''')

Executando query:
('SOM LIVRE',)
('EPIC',)


Quais cds foram lançados na década de 90 e custam 10 reais ou menos?

In [71]:
# YEAR(Data_Lancamento) BETWEEN 1990 AND 1999
db('''
SELECT 
    Nome_CD 
FROM 
    CD
WHERE 
    YEAR(Data_Lancamento) >= 1990 AND YEAR(Data_Lancamento) < 2000 AND Preco_venda <= 10 
''')

Executando query:
('Barry Manilow Greatest Hits Vol I',)
('Listen Without Prejudice',)


Quais cds receberam alguma recomendação? Liste seus ids.

In [72]:
#DISTINCT evita repetições
db('''
SELECT DISTINCT 
    CD_Indicado 
FROM 
    CD
''')

Executando query:
(None,)
(1,)
(2,)
(3,)
(5,)
(7,)


## Agregação

Quantas músicas tem na base?

In [73]:
db('SELECT COUNT(*) FROM MUSICA')

Executando query:
(88,)


Note o comando `COUNT` acima. Trata-se de um comando de *agregação*: uma operação que trabalha em cima de todo o resultado da query e retorna um valor agregado para esses resultados. 

Usamos o argumento "\*" para indicar que queremos apenas contar quantas linhas não são nulas. Normalmente é melhor especificar direito a coluna sobre a qual queremos operar.

Por exemplo: qual a duração da musica mais longa?

In [74]:
db('SELECT MAX(duracao) FROM MUSICA')

Executando query:
(9,)


**Atividade:** Quantas musicas foram escritas pelo autor número 1 (Renato Russo)?

In [75]:
db('''
SELECT 
    COUNT(*)
FROM 
    MUSICA_AUTOR 
WHERE 
    Codigo_Autor = 1''')

Executando query:
(15,)


**Atividade:** Se eu fosse comprar uma cópia de cada CD da base, quanto gastaria?

In [76]:
db('SELECT SUM(Preco_Venda) FROM CD')

Executando query:
(Decimal('84'),)


## Variáveis

Verificando manualmente a tabela `musica` podemos encontrar o nome da musica mais longa:

In [77]:
db('SELECT * from MUSICA')

Executando query:
(1, 'Será', 2)
(2, 'Ainda é Cedo', 4)
(3, 'Geração Coca-Cola', 2)
(4, 'Eduardo e Monica', 4)
(5, 'Tempo Perdido', 5)
(6, 'Índios', 4)
(7, 'Que País é Este', 3)
(8, 'Faroeste Caboclo', 9)
(9, 'Há Tempos', 3)
(10, 'Pais e Filhos', 5)
(11, 'Meninos e Meninas', 3)
(12, 'Vento no Litoral', 6)
(13, 'Perfeição', 4)
(14, 'Giz', 3)
(15, 'Dezesseis', 5)
(16, 'Antes das Seis', 3)
(17, 'Meninos, Eu Vi', 3)
(18, 'Eu Te Amo', 3)
(19, 'Piano na Mangueira', 2)
(20, 'A Violeira', 3)
(21, 'Anos Dourados', 3)
(22, 'Olha, Maria', 4)
(23, 'Biscate', 3)
(24, 'Retrato em Preto e Branco', 3)
(25, 'Falando de Amor', 3)
(26, 'Pois É', 2)
(27, 'Noite dos Mascarados', 2)
(28, 'Sabiá', 3)
(29, 'Imagina', 3)
(30, 'Bate-Boca', 4)
(31, 'Cai Dentro', 2)
(32, 'O Bêbado e o Equilibrista', 3)
(33, 'Essa Mulher', 3)
(34, 'Basta de Clamares Inocência', 3)
(35, 'Beguine Dodói', 2)
(36, 'Eu hein Rosa', 3)
(37, 'Altos e Baixos', 3)
(38, 'Bolero de Satã', 3)
(39, 'Pé Sem Cabeça', 3)
(40, 'As Aparências Engana

Nenhuma surpresa aqui: é "Faroeste Caboclo"...

Agora suponha que eu quero localizar o *nome* da música mais comprida. Note que o comando a seguir NÃO FAZ SENTIDO (porque?):

```SQL
SELECT Nome_musica, MAX(duracao) FROM MUSICA;
```

Talvez se especificássemos que a duracao da música tem que coincidir com a duracao máxima...

In [78]:
try:
    db('SELECT Nome_musica FROM MUSICA WHERE duracao=MAX(duracao)')
except mysql.connector.DatabaseError as e:
    print(f'DatabaseError: {e}')

Executando query:
DatabaseError: 1111 (HY000): Invalid use of group function


Também não funciona: não podemos usar uma função de agrupamento dentro da cláusula `WHERE`. Como resolver isso então?

Primeiro descubra qual é a maior duração e guarde em uma *variável* ***no banco de dados***.

In [79]:
db('SELECT MAX(duracao) INTO @max_duracao from MUSICA')

Executando query:


Podemos checar o valor dessa variavel agora:

In [80]:
db('SELECT @max_duracao')

Executando query:
(9,)


Agora podemos terminar o serviço!

In [81]:
db('SELECT Nome_musica FROM MUSICA WHERE duracao=@max_duracao')

Executando query:
('Faroeste Caboclo',)


**Atividade**: Encontre todas as musicas com duração acima da média.

In [82]:
db('SELECT AVG(duracao) INTO @media_duracao from MUSICA')

Executando query:


In [83]:
db('SELECT @media_duracao')

Executando query:
(Decimal('3.284090909'),)


In [84]:
db('SELECT Nome_musica FROM MUSICA WHERE duracao>=@media_duracao')

Executando query:
('Ainda é Cedo',)
('Eduardo e Monica',)
('Tempo Perdido',)
('Índios',)
('Faroeste Caboclo',)
('Pais e Filhos',)
('Vento no Litoral',)
('Perfeição',)
('Dezesseis',)
('Olha, Maria',)
('Bate-Boca',)
('As Aparências Enganam',)
('É o Amor',)
('Devolva-me',)
('Inverno',)
('Vambora',)
('Maresia',)
('Naquela Estação',)
('New York City Rhythm',)
("It's a Miracle",)
('Trying to get the feeling again',)
('Some Kind of Friend',)
('Praying for Time',)
('Freedom 90',)
('Something to Save',)
('Cowboys and Angels',)


## PAREI AQUIIIIIIIIIIIIIIIII AAAAAAAAAAAAAAAAAAA

## Pesquisa em múltiplas tabelas

### Produto cartesiano

Quando listamos várias tabelas no comando `SELECT` temos como resultado o produto cartesiano destas. 

(Para os matemáticos: não se trata estritamente do produto cartesiano, pois não geramos um conjunto de duplas de tuplas. Geramos um "aplainamento" - *flattening* - do resultado, para cada linha.)

Por exemplo, se temos uma tabela com 2 linhas e outra com 3 linhas, o resultado terá 6 linhas, como ilustrado a seguir:

```SQL
DROP DATABASE IF EXISTS test;
CREATE DATABASE test;
USE test;

CREATE TABLE tab1 (
    coluna11 INT,
    coluna21 INT
);

CREATE TABLE tab2 (
    coluna12 INT,
    coluna22 INT
);

INSERT INTO tab1 VALUES (1, 2), (3, 4);
INSERT INTO tab2 VALUES (1000, 2000), (3000, 4000), (5000, 6000);

SELECT * FROM tab1, tab2;
```

Resultado:

![Produto cartesiano](imgs/cartesiano.png)

Observe o que acontece quando unimos as tabelas *cd* e *gravadora*

In [85]:
db('SELECT * FROM CD')

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)


In [86]:
db('SELECT * FROM GRAVADORA')

Executando query:
(1, 'EMI', 'Rod. Pres. Dutra, s/n km 229,8', None, 'JOÃO', 'www.emi-music.com.br')
(2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(3, 'SOM LIVRE', None, None, 'MARTA', 'www.somlivre.com.br')
(4, 'EPIC', None, None, 'PAULO', 'www.epic.com.br')


In [87]:
db('DESCRIBE CD')

Executando query:
('Codigo_CD', b'int', 'NO', 'PRI', None, '')
('Codigo_Gravadora', b'int', 'YES', 'MUL', None, '')
('Nome_CD', b'varchar(60)', 'YES', '', None, '')
('Preco_Venda', b'int', 'YES', '', None, '')
('Data_Lancamento', b'date', 'YES', '', None, '')
('CD_Indicado', b'int', 'YES', 'MUL', None, '')


In [88]:
db('DESCRIBE GRAVADORA')

Executando query:
('Codigo_Gravadora', b'int', 'NO', 'PRI', None, '')
('Nome_Gravadora', b'varchar(60)', 'YES', '', None, '')
('Endereco', b'varchar(60)', 'YES', '', None, '')
('Telefone', b'varchar(20)', 'YES', '', None, '')
('Contato', b'varchar(20)', 'YES', '', None, '')
('URL', b'varchar(80)', 'YES', '', None, '')


In [89]:
db('SELECT * FROM CD, GRAVADORA')

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 4, 'EPIC', None, None, 'PAULO', 'www.epic.com.br')
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 3, 'SOM LIVRE', None, None, 'MARTA', 'www.somlivre.com.br')
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 1, 'EMI', 'Rod. Pres. Dutra, s/n km 229,8', None, 'JOÃO', 'www.emi-music.com.br')
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 4, 'EPIC', None, None, 'PAULO', 'www.epic.com.br')
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 3, 'SOM LIVRE', None, None, 'MARTA', 'www.somlivre.com.br')
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 1, 'EMI', 'Rod. Pres. Dutra, s/n km 229,8', None, 'JOÃO', 'www.emi-music.c

In [90]:
db('SELECT COUNT(*) FROM CD, GRAVADORA')

Executando query:
(28,)


In [91]:
db('SELECT COUNT(*) FROM CD')

Executando query:
(7,)


In [92]:
db('SELECT COUNT(*) FROM GRAVADORA')

Executando query:
(4,)


### Inner join

E se filtrássemos o resultado anterior, mantendo apenas as linhas em que o código de gravadora na tabela *'cd'* coincida com o código de gravadora na tabela *'gravadora'*?

In [93]:
db('''
SELECT 
    * 
FROM 
    CD as c, GRAVADORA as g 
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
''')

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 1, 'EMI', 'Rod. Pres. Dutra, s/n km 229,8', None, 'JOÃO', 'www.emi-music.com.br')
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1, 3, 'SOM LIVRE', None, None, 'MARTA', 'www.somlivre.com.br')
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2, 3, 'SOM LIVRE', None, None, 'MARTA', 'www.somlivre.com.br')
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')


In [94]:
db('''
SELECT 
    Nome_CD, Nome_Gravadora 
FROM
    CD as c, GRAVADORA as g 
WHERE
    c.Codigo_Gravadora = g.Codigo_Gravadora
''')

Executando query:
('Mais do Mesmo', 'EMI')
('Bate-Boca', 'BMG')
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('A Força que nunca Seca', 'BMG')
('Perfil', 'SOM LIVRE')
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Listen Without Prejudice', 'BMG')


Opa, parece que com isso conseguimos associar a gravadora ao cd!

Estamos mantendo apenas as linhas do produto cartesiano onde o codigo da gravadora declarado na tabela cd bate com o codigo da gravadora declarado na tabela gravadora. Logo, nestas linhas, a informação proveniente da tabela cd e a informação proveniente da tabela gravadora estarão se referindo à mesma gravadora. Conseguimos, portanto, conectar as informações de cada CD com as informações da gravadora à qual aquele CD pertence.

Denominamos *"inner join"*, *"equi-join"* ou "união regular" a essa operação de junção de tabelas através da união de chave primária de uma com chave estrangeira de outra. Eu chamo informalmente de "vamos bater chaves!"

Vamos construir uma lista de cds e suas respectivas gravadoras:

In [95]:
db('''
SELECT 
    Nome_CD, Nome_Gravadora 
FROM 
    CD as c, GRAVADORA as g 
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
''')

Executando query:
('Mais do Mesmo', 'EMI')
('Bate-Boca', 'BMG')
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('A Força que nunca Seca', 'BMG')
('Perfil', 'SOM LIVRE')
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Listen Without Prejudice', 'BMG')


Vamos aprimorar nossa query para procurar apenas os CDs gravados pela 'SOM LIVRE':

In [96]:
db("""
SELECT 
    Nome_CD, Nome_Gravadora 
FROM 
    CD as c, 
    GRAVADORA as g 
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
    AND g.Nome_Gravadora = 'SOM LIVRE'
""")

Executando query:
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('Perfil', 'SOM LIVRE')


Existem "apelidos" para a união regular, que produzem o mesmo resultado:

In [97]:
db('''
SELECT 
    Nome_CD, 
    Nome_Gravadora 
FROM 
    CD 
    INNER JOIN GRAVADORA USING (Codigo_Gravadora)
''')

Executando query:
('Mais do Mesmo', 'EMI')
('Bate-Boca', 'BMG')
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('A Força que nunca Seca', 'BMG')
('Perfil', 'SOM LIVRE')
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Listen Without Prejudice', 'BMG')


In [98]:
db('''
SELECT
    Nome_CD,
    Nome_Gravadora
FROM
    CD
    INNER JOIN GRAVADORA ON CD.Codigo_Gravadora = GRAVADORA.Codigo_Gravadora
''')

Executando query:
('Mais do Mesmo', 'EMI')
('Bate-Boca', 'BMG')
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('A Força que nunca Seca', 'BMG')
('Perfil', 'SOM LIVRE')
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Listen Without Prejudice', 'BMG')


In [99]:
db("""
SELECT 
    Nome_CD, Nome_Gravadora 
FROM 
    CD as c, 
    GRAVADORA as g 
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
    AND g.Nome_Gravadora = 'SOM LIVRE'
""")

Executando query:
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('Perfil', 'SOM LIVRE')


**Atividade**: Construa a lista das musicas do Renato Russo.

In [100]:
db("""
SELECT 
    MUSICA.Nome_Musica
FROM 
    AUTOR, MUSICA_AUTOR, MUSICA
WHERE
    AUTOR.Codigo_Autor = MUSICA_AUTOR.Codigo_Autor
    AND MUSICA_AUTOR.Codigo_Musica = MUSICA.Codigo_Musica
    AND AUTOR.Nome_Autor = 'Renato Russo'
""")

Executando query:
('Será',)
('Geração Coca-Cola',)
('Eduardo e Monica',)
('Tempo Perdido',)
('Índios',)
('Que País é Este',)
('Faroeste Caboclo',)
('Há Tempos',)
('Pais e Filhos',)
('Meninos e Meninas',)
('Vento no Litoral',)
('Perfeição',)
('Giz',)
('Dezesseis',)
('Antes das Seis',)


In [101]:
db("""
SELECT 
    MUSICA.Nome_Musica
FROM 
    AUTOR
    INNER JOIN MUSICA_AUTOR ON AUTOR.Codigo_Autor = MUSICA_AUTOR.Codigo_Autor
    INNER JOIN MUSICA ON MUSICA_AUTOR.Codigo_Musica = MUSICA.Codigo_Musica
WHERE
    AUTOR.Nome_Autor = 'Renato Russo'
""")

# BATE CHAVES NA PARTE DO FROM

Executando query:
('Será',)
('Geração Coca-Cola',)
('Eduardo e Monica',)
('Tempo Perdido',)
('Índios',)
('Que País é Este',)
('Faroeste Caboclo',)
('Há Tempos',)
('Pais e Filhos',)
('Meninos e Meninas',)
('Vento no Litoral',)
('Perfeição',)
('Giz',)
('Dezesseis',)
('Antes das Seis',)


In [102]:
db("""
SELECT 
    MUSICA.Nome_Musica
FROM 
    AUTOR
    INNER JOIN MUSICA_AUTOR USING (Codigo_Autor)
    INNER JOIN MUSICA USING (Codigo_Musica)
WHERE
    AUTOR.Nome_Autor = 'Renato Russo'
""")

# APENAS QUANDO É O MESMO NOME DO CAMPO

Executando query:
('Será',)
('Geração Coca-Cola',)
('Eduardo e Monica',)
('Tempo Perdido',)
('Índios',)
('Que País é Este',)
('Faroeste Caboclo',)
('Há Tempos',)
('Pais e Filhos',)
('Meninos e Meninas',)
('Vento no Litoral',)
('Perfeição',)
('Giz',)
('Dezesseis',)
('Antes das Seis',)


### Self join

E se quisermos listar os nomes de CDs e seus respectivos CDs indicados? Veja a solução abaixo. Note o uso de dois *aliases* diferentes para a tabela cd.

In [103]:
# Observe também o uso da função CONCAT(), só para ficar mais bonito!
# É um inner join só que da tabela com ela mesma :D
db('''
SELECT
    CONCAT('"', c1.Nome_CD, '" indicado por "', c2.Nome_CD, '"')
FROM
    CD c1,
    CD c2
WHERE
    c1.Codigo_CD = c2.CD_Indicado
''')

Executando query:
('"Perfil" indicado por "Mais do Mesmo"',)
('"Elis Regina - Essa Mulher" indicado por "Bate-Boca"',)
('"Mais do Mesmo" indicado por "Elis Regina - Essa Mulher"',)
('"Mais do Mesmo" indicado por "A Força que nunca Seca"',)
('"Bate-Boca" indicado por "Perfil"',)
('"Listen Without Prejudice" indicado por "Barry Manilow Greatest Hits Vol I"',)


### *'Non-equi-join'*

Chamamos de *'non-equi-join'* a uma junção de tabelas que não usa a igualdade de colunas como critério de junção. 

Podemos fazer muitas investigações úteis com junção de tabelas. Vamos acompanhar um exemplo (também está no livro-texto).

Na base 'musica' temos uma tabela estranha, de poucas linhas, chamada cd_categoria.

In [104]:
db('DESCRIBE CD_CATEGORIA')

Executando query:
('Codigo_Categoria', b'int', 'NO', '', None, '')
('Menor_Preco', b'int', 'NO', '', None, '')
('Maior_Preco', b'int', 'NO', '', None, '')


In [105]:
db('SELECT * from CD_CATEGORIA')

Executando query:
(1, 5, 10)
(2, 10, 12)
(3, 12, 15)
(4, 15, 20)


Esta tabela lista as categorias de preço de CDs. Por exemplo, se um CD custa 9 reais, ele está na categoria 1.

Gostariamos de usar essa informação em conjunto com a tabela de CDs para obter a lista de CDs em cada categoria. Por onde começar?

Vamos montar o produto cartesiano desta tabela cd_categoria com a tabela de CDs e ver se algum *insight* aparece:

In [106]:
db('SELECT * FROM CD, CD_CATEGORIA LIMIT 10')

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 4, 15, 20)
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 3, 12, 15)
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 2, 10, 12)
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 1, 5, 10)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 4, 15, 20)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 3, 12, 15)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 2, 10, 12)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 1, 5, 10)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1, 4, 15, 20)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1, 3, 12, 15)


Interessante! Como se trata do produto cartesiano, cada CD é listado repetidas vezes, uma para cada categoria de CDs. 

*Cada linha tem* ***o preço do CD*** *e um par de valores com* ***limites de preço de faixa***. Leia esta sentença quantas vezes for necessário.

E se filtrássemos esse resultado para manter apenas as linhas onde o preço do CD está dentro dos limites de preço?

In [107]:
# BETWEEN É MAIOR IGUAL DOS DOIS LADOS
db("""
SELECT 
    * 
FROM 
    CD, 
    CD_CATEGORIA cat 
WHERE 
    CD.Preco_Venda BETWEEN cat.Menor_Preco AND cat.Maior_Preco
""")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 4, 15, 20)
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 3, 12, 15)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 3, 12, 15)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 2, 10, 12)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1, 3, 12, 15)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1, 3, 12, 15)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2, 2, 10, 12)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7, 2, 10, 12)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7, 1, 5, 10)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None, 1, 5, 10)


Surgiu um pequeno problema: tem CD alocado para duas categorias ao mesmo tempo! Culpa do operador `BETWEEN` e da nossa interpretação da tabela `cd_categoria`. Vamos remover o `BETWEEN` e usar os limites do jeito que a gente quer:

In [108]:
db("""
SELECT 
    * 
FROM 
    CD, 
    CD_CATEGORIA cat 
WHERE 
    CD.Preco_Venda >= cat.Menor_Preco 
    AND CD.Preco_Venda < cat.Maior_Preco
""")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 4, 15, 20)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 3, 12, 15)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1, 3, 12, 15)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1, 3, 12, 15)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2, 2, 10, 12)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7, 2, 10, 12)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None, 1, 5, 10)


Finalmente, vamos ajustar nossa query para buscar os nomes de CDs na faixa 2 de preço:

In [109]:
db("""
SELECT 
    Nome_CD 
FROM 
    CD, 
    CD_CATEGORIA cat 
WHERE 
    CD.Preco_Venda >= cat.Menor_Preco 
    AND CD.Preco_Venda < cat.Maior_Preco
    AND cat.Codigo_categoria = 2
""")

Executando query:
('Perfil',)
('Barry Manilow Greatest Hits Vol I',)


#### Vamos praticar

Liste o nome do CD e de sua respectiva gravadora para CDs na categoria 2.

In [49]:
#selecione o nome do CD e da gravadora, as quais pertencem a categoria 2 (entre o menor e maior preço e cat 2)

db("""
SELECT 
    Nome_CD, Nome_Gravadora 
FROM 
    CD,
    GRAVADORA as g,
    CD_CATEGORIA cat
WHERE 
    CD.Codigo_Gravadora = g.Codigo_Gravadora
    AND CD.Preco_Venda >= cat.Menor_Preco 
    AND CD.Preco_Venda < cat.Maior_Preco
    AND cat.Codigo_categoria = 2
""")

Executando query:
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Perfil', 'SOM LIVRE')


In [110]:
#selecione o nome do CD e da gravadora, as quais pertencem a categoria 2 (entre o menor e maior preço e cat 2)

db("""
SELECT 
    Nome_CD, Nome_Gravadora 
FROM 
    CD INNER JOIN GRAVADORA USING (Codigo_Gravadora),
    CD_CATEGORIA cat
WHERE 
    CD.Preco_Venda >= cat.Menor_Preco 
    AND CD.Preco_Venda < cat.Maior_Preco
    AND cat.Codigo_categoria = 2
""")

Executando query:
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Perfil', 'SOM LIVRE')


Vamos agora fechar a conexão com este banco de dados, porque no próximo tópico vamos abrir uma conexão com outro banco de dados.

In [111]:
connection.close()

### Outer join

Considere a seguinte base de testes: cada perigo tem um nome e cada comida tem um nome e um perigo associado. (Use o script `tranqueira.sql` para testar essa base.)

```SQL
DROP DATABASE IF EXISTS tranqueira;
CREATE DATABASE tranqueira;
USE tranqueira;

CREATE TABLE comida (
	id INT NOT NULL AUTO_INCREMENT,
	Nome VARCHAR(30),
    idPerigo INT,
    PRIMARY KEY (id)
);

CREATE TABLE perigo (
	id INT NOT NULL AUTO_INCREMENT,
    Nome VARCHAR(20),
    PRIMARY KEY (id)
);

ALTER TABLE comida ADD FOREIGN KEY (idPerigo) REFERENCES perigo (id);

INSERT INTO perigo VALUES (1, 'Cardiaco'), (2, 'Intestinal'), (3, 'Dermatologico'), (4, 'Mental');
INSERT INTO comida VALUES (1, 'Torresmo', 1), (2, 'Alface', NULL), (3, 'Coxinha', 2), (4, 'Espetinho', 2);

SELECT * FROM comida;
SELECT * FROM perigo;
```

![Tabela comida](imgs/comida.png)

![Tabela perigo](imgs/perigo.png)

In [112]:
connection = mysql.connector.connect(
    host='localhost',
    user='megadados',
    password='megadados',
    database='tranqueira',
)

db = partial(run_db_query, connection)

Fazendo o inner join da tabela comida com a tabela perigo temos a lista de comidas que estão associadas a algum perigo:

In [113]:
db('USE tranqueira')
db('''
SELECT 
    comida.Nome, perigo.Nome 
FROM 
    comida INNER JOIN perigo ON comida.idPerigo = perigo.id
''')

Executando query:
Executando query:
('Torresmo', 'Cardiaco')
('Coxinha', 'Intestinal')
('Espetinho', 'Intestinal')


Note que a alface não aparece aqui. 

Se quisermos que todas as comidas apareçam neste join, mesmo que não estejam associadas a um perigo, como fazer? Para isso servem os ***outer joins***. Vamos experimentar com o **`LEFT OUTER JOIN`**:

In [115]:
#Quero listar todas as comidas com ou sem perigo, e para aquelas que tem perigo quero o nome associado

db('''
SELECT 
    comida.Nome, 
    perigo.Nome 
FROM
    comida
    LEFT OUTER JOIN perigo ON comida.idPerigo = perigo.id
''')

#LEFT OUUTER JOIN: O lado esquerdo vai aparecer integralmente, no caso comida. No lado direito aparece nulo ou as conexões.

Executando query:
('Torresmo', 'Cardiaco')
('Alface', None)
('Coxinha', 'Intestinal')
('Espetinho', 'Intestinal')


Veja que agora podemos observar que a alface não oferece perigo! E se quisermos fazer o contrário: queremos que todos os perigos apareçam, mesmo que não exista comida associada? Vamos usar o **`RIGHT OUTER JOIN`**:

In [116]:
db('''
SELECT 
    comida.Nome,
    perigo.Nome
FROM
    comida
    RIGHT OUTER JOIN perigo ON comida.idPerigo = perigo.id
''')

# Agora aparece todos do lado direito, ou seja, os perigos.

Executando query:
('Torresmo', 'Cardiaco')
('Espetinho', 'Intestinal')
('Coxinha', 'Intestinal')
(None, 'Dermatologico')
(None, 'Mental')


Agora os perigos dermatológico e mental aparecem, mas não existem comidas associadas a eles!

Os dois tipos de join são redundantes: basta inverter a ordem das partes para trocar de `RIGHT OUTER JOIN` para `LEFT OUTER JOIN`. Por exemplo:

In [55]:
db('''
SELECT
    comida.Nome,
    perigo.Nome
FROM
    perigo
    LEFT OUTER JOIN comida ON comida.idPerigo = perigo.id
''')

Executando query:
('Torresmo', 'Cardiaco')
('Espetinho', 'Intestinal')
('Coxinha', 'Intestinal')
(None, 'Dermatologico')
(None, 'Mental')


#### Vamos praticar

Liste os perigos que não estão associados a nenhuma comida.

In [118]:
db('''
SELECT 
    perigo.Nome
FROM
    comida
    RIGHT OUTER JOIN perigo ON comida.idPerigo = perigo.id
WHERE 
    comida.idPerigo IS NULL
''')

Executando query:
('Dermatologico',)
('Mental',)


In [119]:
db('''
SELECT 
    perigo.Nome
FROM
    perigo
    LEFT OUTER JOIN comida ON comida.idPerigo = perigo.id
WHERE 
    comida.idPerigo IS NULL
''')

Executando query:
('Dermatologico',)
('Mental',)


## Finalizando o trabalho

Por fim, vamos fechar nossa conexão com o banco de dados!

In [120]:
connection.close()

Se você tentar fechar a conexão duas vezes, nada acontecerá nesta biblioteca. Outras bibliotecas podem lançar terá uma exceção: consulte a documentação da sua biblioteca.

In [121]:
connection.close()

Por hoje é só! Pratique com os exercícios do seu livro texto.